In [2]:
import gzip
from collections import defaultdict
import numpy as np
import urllib
import scipy.optimize
import random
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
import math
import scipy.optimize
import numpy
import pandas as pd
import pickle
from textstat.textstat import textstat
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [3]:
allHelpful = []
userHelpful = defaultdict(list)
item_rate_average=defaultdict(int)
item_rate=defaultdict(list)
user_reviewnum=defaultdict(int)
item_review=defaultdict(str)
item_word=defaultdict(list)
index=0
sum_1=0
for l in readGz("C:\Users\\12815\\Desktop\\Assignment 1\\assignment1\\train.json.gz"):
    user, item, review= l['reviewerID'], l['itemID'], l['reviewText']
    # item_review[item]+=" "+review
    user_reviewnum[user]=user_reviewnum[user]+1
    if l['helpful']['outOf']==1:
        index=index+1
        sum_1=sum_1+l['helpful']['nHelpful']
    item_rate[item].append(l['rating'])
    allHelpful.append(l['helpful'])
    userHelpful[user].append(l['helpful'])
average_1=sum_1*1.0/index
print average_1

0.757906903436


In [4]:
for item in item_rate.keys():
    rate_average=sum(item_rate[item])*1.0/len(item_rate[item])
    item_rate_average[item]=rate_average

averageRate = sum([x['nHelpful'] for x in allHelpful]) * 1.0 / sum([x['outOf'] for x in allHelpful])
userRate = {}
for u in userHelpful:
    totalU = sum([x['outOf'] for x in userHelpful[u]])
    if totalU > 0:
        userRate[u] = sum([x['nHelpful'] for x in userHelpful[u]]) * 1.0 / totalU
    else:
        userRate[u] = averageRate

allHelpful_training = allHelpful
helpfulness_train = []

In [5]:
feat_train= []
rating_list=[]
len_time_list=[]
outOf_list=[]
reviews=[]
helpfulness=[]
len_review_list=[]
pun_ratio_list=[]
len_summary_list=[]
constant_list=[]
rating_sq_list=[]
nhelpful_list=[]
user_num_review_list=[]
for l in readGz("C:\Users\\12815\\Desktop\\Assignment 1\\assignment1\\train.json.gz"):
    user, item = l['reviewerID'], l['itemID']
    rating = (l['rating']-item_rate_average['item'])**2
    # rating= l['rating']
    user_num_review=user_reviewnum[user]
    len_review = textstat.lexicon_count(l['reviewText'])**2
    pun_ratio = 0 if not len_review else (
                                             1.0 * sum([c in string.punctuation for c in l['reviewText']])) / len_review
    len_time = math.log(int(l['unixReviewTime'] / 100000) + 1)
    outOf = int(l['helpful']['outOf'])
    pun_ratio_list.append(pun_ratio)
    len_review_list.append(len_review)
    rating_list.append(rating)
    user_num_review_list.append(user_num_review)
    len_time_list.append(len_time)
    outOf_list.append(outOf)
    reviews.append(l['reviewText'])
    helpfulness.append(averageRate if (not l['helpful']['outOf']) else l['helpful']['nHelpful']*1.0/l['helpful']['outOf'])
    nhelpful_list.append(l['helpful']['nHelpful'])
reviewData = {'reviewText': reviews,
              'rating':rating_list,
              'len_time': len_time_list,
              'outOf':outOf_list,
              'helpfulness': helpfulness,
              'len_review': len_review_list,
              'pun_ratio':pun_ratio_list,
              'nHelpful': nhelpful_list,
              'user_num_review':user_num_review_list
              }
df_reviews=pd.DataFrame(reviewData)

In [25]:
df_reviews=pd.DataFrame(reviewData)

from textblob import TextBlob
# df_reviews['polarity'] = df_reviews['reviewText'].apply(lambda x: TextBlob(x).sentiment.polarity)
df_reviews['len_sentences'] = df_reviews['reviewText'].apply(lambda x: len(TextBlob(x).sentences))
# df_reviews['subjectivity'] = df_reviews['reviewText'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
df_train, df_validate=train_test_split(df_reviews, test_size=0.05)
df_train=df_reviews
df_validate=df_reviews[190000:]
valid_targets_nHelpful = df_validate['nHelpful'].values
valid_targets_outof=df_validate['outOf'].values
del df_validate['reviewText']
del df_validate['helpfulness']
del df_validate['nHelpful']
validate=np.nan_to_num(np.array(df_validate))
print "Length of validate set",len(validate)
print "Validating Set done"




train_targets = df_train['helpfulness'].values
df_train_1=df_train[df_train.outOf > 10] #linear regression
# df_train_2=df_train[df_train.outOf <=4 and df_train.outOf>=2] #latent factor
df_train_3=df_train[df_train.outOf ==1 ] #SVM
train_targets_1 = df_train_1['helpfulness'].values
# train_targets_2= df_train_2['helpfulness'].values
train_targets_3= df_train_3['helpfulness'].values

del df_train_1['helpfulness']
del df_train_1['reviewText']
del df_train_1['nHelpful']
# del df_train_2['helpfulness']
# del df_train_2['reviewText']
# del df_train_2['nHelpful']
del df_train_3['helpfulness']
del df_train_3['reviewText']
del df_train_3['nHelpful']

train_1=np.nan_to_num(np.array(df_train_1))
# train_2=np.nan_to_num(np.array(df_train_2))
train_3=np.nan_to_num(np.array(df_train_3))

print "Length of train set(outOf>4)",len(train_1)
print "Training Set done(outOf>4)"
# print "Length of train set(2<=outOf<=4)",len(train_2)
# print "Training Set done(2<=outOf<=4)"
print "Length of train set(outOf==1)",len(train_3)
print "Training Set done(outOf==1)"




userHelpful_test = defaultdict(list)
for l in readGz("C:\Users\\12815\\Desktop\\Assignment 1\\assignment1\\test_Helpful.json.gz"):

    user, item = l['reviewerID'], l['itemID']
    rating=(l['rating']-item_rate_average['item'])**2
    len_time=math.log(int(l['unixReviewTime']/100000)+1)
    outOf=int(l['helpful']['outOf'])
    len_review = textstat.lexicon_count(l['reviewText'])**2
    pun_ratio = 0 if not len_review else (1.0 * sum([c in string.punctuation for c in l['reviewText']])) / len_review
    userHelpful_test['user'].append(user)
    userHelpful_test['item'].append(item)
    userHelpful_test['rating'].append(rating)
    userHelpful_test['len_time'].append(len_time)
    userHelpful_test['outOf'].append(outOf)
    userHelpful_test['reviewText'].append(l['reviewText'])
    userHelpful_test['len_review'].append(len_review)
    userHelpful_test['pun_ratio'].append(pun_ratio)
    userHelpful_test['user_num_review'].append(user_reviewnum[user])
reviewData_test= {'reviewText':userHelpful_test['reviewText'],
                  'rating':userHelpful_test['rating'],
               'len_time': userHelpful_test['len_time'],
              'outOf':userHelpful_test['outOf'],
                  'len_review ':userHelpful_test['len_review'],
                  'pun_ratio ':userHelpful_test['pun_ratio'],
                'user_num_review':userHelpful_test['user_num_review']
                 }
df_reviews_test=pd.DataFrame(reviewData_test)
#
from textblob import TextBlob
# df_reviews_test['polarity'] = df_reviews_test['reviewText'].apply(lambda x: TextBlob(x).sentiment.polarity)
df_reviews_test['len_sentences'] = df_reviews_test['reviewText'].apply(lambda x: len(TextBlob(x).sentences))
# df_reviews_test['subjectivity'] = df_reviews_test['reviewText'].apply(lambda x: TextBlob(x).sentiment.subjectivity)
# #
del df_reviews_test['reviewText']
test=np.nan_to_num(np.array(df_reviews_test))
print "length of Test Set:", len(test)
print "Test Set done"

Length of validate set 10000
Validating Set done
Length of train set(outOf>4) 4669
Training Set done(outOf>4)
Length of train set(outOf==1) 28551
Training Set done(outOf==1)
length of Test Set: 14000
Test Set done


In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import Ridge
# Train Set 1
model=Ridge()
alpha_can=np.logspace(-3,2,10)
ridge_mode1 = GridSearchCV(model, param_grid={'alpha': alpha_can}, cv=5)
ridge_mode1.fit(train_1, train_targets_1)

# Train Set 2
user_item = defaultdict(list)
item_user= defaultdict(list)
pair_helpfulness=defaultdict(list)
index=0
for l in readGz("C:\Users\\12815\\Desktop\\Assignment 1\\assignment1\\train.json.gz"):
    if(l['helpful']['outOf']<=2):
        continue
    user, item = l['reviewerID'], l['itemID']
    user_item[user].append(item)
    item_user[item].append(user)
    pair_helpfulness[user + item].append(averageRate if (not l['helpful']['outOf']) else l['helpful']['nHelpful']*1.0/l['helpful']['outOf'])
    index=index+1
lamda=2
alpha=averageRate
alpha_prev=0
beta_user=defaultdict(int)
beta_item=defaultdict(int)
time=0
while(time<300):
    time=time+1
    alpha_prev=alpha
    for user in user_item.keys():
        beta_user[user]=sum((pair_helpfulness[user+x][0]-alpha_prev-beta_item[x]+0.0) for x in user_item[user])/(lamda+len(user_item[user]))
    for item in item_user.keys():
        beta_item[item]=sum((pair_helpfulness[x+item][0]-alpha_prev-beta_user[x]+0.0) for x in item_user[item])/(lamda+len(item_user[item]))
    sum_temp=0
    for user in user_item.keys():
        for item in user_item[user]:
            sum_temp=sum_temp+(pair_helpfulness[user+item][0]-beta_item[item]-beta_user[user])
    alpha=sum_temp/index
    if(alpha-alpha_prev<0.0001):
        print " break"
        break
# param_test = {'n_estimators':range(20,81,10)}
# gsearch = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.1, min_samples_split=500,min_samples_leaf=50,max_depth=8,max_features='sqrt',subsample=0.8,random_state=10), 
# param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
# gsearch.fit(train[predictors],train[target])


# Train Set 3
# LR_model=LogisticRegression()
# LR_model.fit(train_3, list(train_targets_3))
# 7 0.162
# 6 0.1608
# 5 0.1574
# 4  0.1534
# 3  0.148
# 2  0.142
print "Training Done!"

 break
Training Done!


In [30]:
index=0
t=0
mae=0
for l in readGz("C:\Users\\12815\\Desktop\\Assignment 1\\assignment1\\train.json.gz"):
    if(index<190000):
        index=index+1
        continue
    user, item = l['reviewerID'], l['itemID']
    outOf=l['helpful']['outOf']
    outOf=int(outOf)
    if outOf>6:
        helpfulness_temp = ridge_mode1.predict(validate[t,:].reshape(1,-1))[0]
        mae+=abs(helpfulness_temp*outOf-valid_targets_nHelpful[t])
    elif outOf==1:
        # helpfulness_temp= LR_model.predict_proba(validate[t,:].reshape(1,-1))[0][1]
        # print helpfulness_temp
        # mae+=abs(round(helpfulness_temp*outOf)-valid_targets_nHelpful[t])
        helpfulness_temp = alpha + beta_user[user] + beta_item[item]
        print helpfulness_temp
        mae += abs(helpfulness_temp * outOf - valid_targets_nHelpful[t])
    else:
        helpfulness_temp=alpha+beta_user[user]+beta_item[item]
        mae+=abs(helpfulness_temp*outOf-valid_targets_nHelpful[t])
    index=index+1
    t=t+1
print "lamda:", lamda
print "mae of validate set:",mae*1.0/len(valid_targets_outof)

0.843512812599
0.900123894028
0.840406353379
0.87325166587
0.827142664355
0.894134778123
0.828857000345
0.860606829693
0.69051658754
0.88833239622
0.928898242674
0.923739686713
0.85328857964
0.820852348454
0.849725731039
0.849725731039
0.862031331118
0.810203526626
0.849725731039
0.808546748376
0.788816042209
0.818184131157
0.849725731039
0.897516824203
0.905866507398
0.859813716264
0.917118546921
0.898542477847
0.877339168096
0.783297568214
0.905954348263
0.893627818828
0.899068368155
0.632478442955
0.937422725131
0.890520267174
0.849725731039
0.815735034822
0.864102303931
0.84770633557
0.849725731039
0.808169298533
0.845468886772
0.840406353379
0.898887195747
0.902174827375
0.860398600227
0.900562640028
0.886732708876
0.877429069535
0.714495950613
0.855375590669
0.840406353379
0.893929725305
0.873175972705
0.874397049597
0.838176378005
0.891063716264
0.931963459565
0.833086126199
0.849725731039
0.902393328813
0.851449320536
0.827065266894
0.838285938486
0.888144534733
0.986615339486


In [32]:
predictions = open("C:\Users\\12815\\Desktop\\Assignment 1\\assignment1\\predictions_Helpful.txt", 'w')
index=0
for l in open("C:\Users\\12815\\Desktop\\Assignment 1\\assignment1\\pairs_Helpful.txt"):
    if l.startswith("userID"):
        # header
        predictions.write(l)
        continue
    # feat_temp=[1.0]
    u, i, outOf = l.strip().split('-')
    outOf = int(outOf)
    if outOf>6:
        helpfulness_temp = ridge_mode1.predict(test[index,:].reshape(1,-1))[0]
    elif outOf==1:
        # helpfulness_temp= 1 if (LR_model.predict_proba(test[index,:].reshape(1,-1))[0][1]>0.8) else 0
        helpfulness_temp = alpha + beta_user[user] + beta_item[item]
    else:
        helpfulness_temp=alpha+beta_user[user]+beta_item[item]
    if(helpfulness_temp<=0):
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(0) + '\n')
    if(outOf*helpfulness_temp>=outOf):
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf) + '\n')
    elif(outOf>2 and outOf-outOf*helpfulness_temp<1):
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf-1) + '\n')
    else:
        predictions.write(u + '-' + i + '-' + str(outOf) + ',' + str(outOf*helpfulness_temp) + '\n')
    index=index+1
predictions.close()
print "done"

done
